In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [2]:
# read in data from July, 28th, 2012
trajectories28 = pd.read_csv("TrajDataV2_20120728.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
# read in data from July, 29th, 2012
trajectories29 = pd.read_csv("TrajDataV2_20120729.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
trajectories28.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
0,845,28,0,1,470.31621,39000,40.692500,-74.168667
1,845,28,0,2,470.31621,39000,40.784686,-74.019193
2,845,28,0,3,470.31621,39000,40.877281,-73.868705
3,845,28,0,4,470.31621,39000,40.969822,-73.717979
4,845,28,0,5,470.31621,39000,41.062186,-73.566848


In [3]:
# concatenate data from both days
#trajectories = pd.concat([trajectories28, trajectories29])
trajectories = trajectories29

In [5]:
# add consecutive flight index to the data
flightNames = trajectories['flight'].unique()
trajectories['flightIndex'] = trajectories['flight'].map(lambda x: np.where(flightNames==x)[0][0])
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude,flightIndex
0,805,29,0,2,465.28117,33000,40.692500,-74.168667,0
1,805,29,0,3,465.28117,33000,40.785863,-74.013012,0
2,805,29,0,4,465.28117,33000,40.878762,-73.856473,0
3,805,29,0,5,465.28117,33000,40.971273,-73.699485,0
4,805,29,0,6,465.28117,33000,41.063590,-73.542087,0


In [6]:
# set consecutive flight index as dataset index
trajectories = trajectories.set_index('flightIndex')
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
flightIndex,,,,,,,,
0,805,29,0,2,465.28117,33000,40.692500,-74.168667
0,805,29,0,3,465.28117,33000,40.785863,-74.013012
0,805,29,0,4,465.28117,33000,40.878762,-73.856473
0,805,29,0,5,465.28117,33000,40.971273,-73.699485
0,805,29,0,6,465.28117,33000,41.063590,-73.542087


In [6]:
trajectories.time.plot(marker='o', linestyle='o')
plt.show()

In [7]:
def plotTrajectories(trajectories):
    # Create a figure of size (i.e. pretty big)
    fig = plt.figure(figsize=(20,10))

    # Create a map, using the Gall–Peters projection, 
    map = Basemap(projection='gall', 
                  # with low resolution,
                  resolution = 'l', 
                  # And threshold 100000
                  area_thresh = 100000.0,
                  # Centered at 0,0 (i.e null island)
                  lat_0=0, lon_0=0)

    # Draw the coastlines on the map
    map.drawcoastlines()

    # Draw country borders on the map
    map.drawcountries()

    # Fill the land with grey
    map.fillcontinents(color = '#888888')

    # Draw the map boundaries
    map.drawmapboundary(fill_color='#f4f4f4')

    # Define our longitude and latitude points
    # We have to use .values because of a wierd bug when passing pandas data
    # to basemap.
    x,y = map(trajectories['longitude'].values, trajectories['latitude'].values)

    # Plot them using round markers of size 6
    map.plot(x, y, 'b', markersize=6)

    # Show the map
    plt.show()

In [84]:
plotTrajectories(trajectories[trajectories.index < 100])

In [8]:
def distance(lat1, lon1, lat2, lon2, R=6367):
    """Get the distance on a great circle between to trajectory points in kilometers
    
    Arguments:
    R: Radius in kilometers
    lat1: latitude of first point in degrees
    lon1: longitude of the first point in degrees
    lat2: latitude of the second point in degrees
    lon2: longitude of the second point in degrees
    
    """
    Lat0 = np.radians(lat1)
    Latf = np.radians(lat2)
    Lon0 = np.radians(lon1)
    Lonf = np.radians(lon2)

    return R * np.arccos(np.sin(Lat0) * np.sin(Latf) + np.cos(Lonf-Lon0)*np.cos(Lat0)*np.cos(Latf))

In [9]:
# test
i1 = 4
i2 = 7
lat1 = trajectories['latitude'].iloc[i1]
lat2 = trajectories['latitude'].iloc[i2]
lon1 = trajectories['longitude'].iloc[i1]
lon2 = trajectories['longitude'].iloc[i2]
distance(lat1, lon1, lat2, lon2)

49.866839043922198

In [10]:
# constants 
# nautic mile in kilometers
nautic = 1.852

In [11]:
# minimal acceptable distance in kilometers
mindistance = 30 * nautic

In [12]:
# minimal acceptable time difference
mintime = 3

In [13]:
def detectSpatialConflict(lat1, lon1, lat2, lon2, mindistance):
    return distance(lat1, lon1, lat2, lon2) < mindistance

In [14]:
t = trajectories[trajectories['time'] == 19]
t

,flight,date,wind,time,speed,altitude,latitude,longitude
flightIndex,,,,,,,,
0,805,29,0,19,465.28117,33000,42.215822,-71.450293
1,806,29,0,19,470.31621,37000,42.317306,-71.434381
2,808,29,0,19,484.18088,35000,42.085812,-71.354173
3,811,29,0,19,458.84508,37000,42.023049,-72.031752
4,812,29,0,19,476.91320,33000,42.368452,-85.265022
5,813,29,0,19,481.78734,37000,41.799785,-71.854765
6,815,29,0,19,458.84508,37000,41.331577,-73.027464
7,816,29,0,19,478.41682,35000,43.003071,-69.818878
8,817,29,0,19,465.28117,33000,41.566298,-72.553204


In [15]:
import itertools

In [16]:
flightPairs = np.array(list(itertools.combinations(t.index, 2)))

In [17]:
t['latitude'][3]

42.023049

In [18]:
df = pd.DataFrame(flightPairs, columns=['first', 'second'])
df.head()

,first,second
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [19]:
df['isConflict'] = df.apply(lambda x: 
         detectSpatialConflict(
        t['latitude'][x['first']], 
        t['longitude'][x['first']], 
        t['latitude'][x['second']], 
        t['longitude'][x['second']], mindistance)
         , axis=1)

In [20]:
df.head()

,first,second,isConflict
0,0,1,True
1,0,2,True
2,0,3,True
3,0,4,False
4,0,5,False


In [21]:
grouped = trajectories.groupby('time')

In [19]:
trajectories.to_hdf('trajectories.h5', 'trajectories', mode='w')

In [20]:
tra = pd.read_hdf('trajectories.h5', 'trajectories')

In [21]:
tra.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
flightIndex,,,,,,,,
0,805,29,0,2,465.28117,33000,40.692500,-74.168667
0,805,29,0,3,465.28117,33000,40.785863,-74.013012
0,805,29,0,4,465.28117,33000,40.878762,-73.856473
0,805,29,0,5,465.28117,33000,40.971273,-73.699485
0,805,29,0,6,465.28117,33000,41.063590,-73.542087


In [24]:
np.array(10, dtype=int)

array(10)